In [5]:
!pip install Catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 1.4 MB/s 


In [6]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
import xgboost
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [7]:
xgboost.__version__

'0.90'

In [8]:
df = pd.read_csv('justice.csv')

In [9]:
df.head()

,Unnamed: 0,ID,name,href,docket,term,first_party,second_party,facts,facts_len,majority_vote,minority_vote,first_party_winner,decision_type,disposition,issue_area
0,0,50606,Roe v. Wade,https://api.oyez.org/cases/1971/70-18,70-18,1971,Jane Roe,Henry Wade,"<p>In 1970, Jane Roe (a fictional name used in...",501,7,2,True,majority opinion,reversed,NaN
1,1,50613,Stanley v. Illinois,https://api.oyez.org/cases/1971/70-5014,70-5014,1971,"Peter Stanley, Sr.",Illinois,<p>Joan Stanley had three children with Peter ...,757,5,2,True,majority opinion,reversed/remanded,Civil Rights
2,2,50623,Giglio v. United States,https://api.oyez.org/cases/1971/70-29,70-29,1971,John Giglio,United States,<p>John Giglio was convicted of passing forged...,495,7,0,True,majority opinion,reversed/remanded,Due Process
3,3,50632,Reed v. Reed,https://api.oyez.org/cases/1971/70-4,70-4,1971,Sally Reed,Cecil Reed,"<p>The Idaho Probate Code specified that ""male...",378,7,0,True,majority opinion,reversed/remanded,Civil Rights
4,4,50643,Miller v. California,https://api.oyez.org/cases/1971/70-73,70-73,1971,Marvin Miller,California,"<p>Miller, after conducting a mass mailing cam...",305,5,4,True,majority opinion,vacated/remanded,First Amendment


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3303 entries, 0 to 3302
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          3303 non-null   int64 
 1   ID                  3303 non-null   int64 
 2   name                3303 non-null   object
 3   href                3303 non-null   object
 4   docket              3303 non-null   object
 5   term                3303 non-null   object
 6   first_party         3302 non-null   object
 7   second_party        3302 non-null   object
 8   facts               3303 non-null   object
 9   facts_len           3303 non-null   int64 
 10  majority_vote       3303 non-null   int64 
 11  minority_vote       3303 non-null   int64 
 12  first_party_winner  3288 non-null   object
 13  decision_type       3296 non-null   object
 14  disposition         3231 non-null   object
 15  issue_area          3161 non-null   object
dtypes: int64(5), object(11)


In [13]:
data = df[['facts', 'issue_area', 'first_party_winner']]

In [14]:
data.head()

,facts,issue_area,first_party_winner
0,"<p>In 1970, Jane Roe (a fictional name used in...",NaN,True
1,<p>Joan Stanley had three children with Peter ...,Civil Rights,True
2,<p>John Giglio was convicted of passing forged...,Due Process,True
3,"<p>The Idaho Probate Code specified that ""male...",Civil Rights,True
4,"<p>Miller, after conducting a mass mailing cam...",First Amendment,True


In [15]:
data = data.dropna()

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3159 entries, 1 to 3302
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   facts               3159 non-null   object
 1   issue_area          3159 non-null   object
 2   first_party_winner  3159 non-null   object
dtypes: object(3)
memory usage: 98.7+ KB


In [17]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
def remove_stopwords(row):
    sentence = row['facts']
    cleaned = []
    sentence = re.sub(r'</*[a-z]+>', '', sentence)
    sentence = re.sub(r'\n', '', sentence)
    for punct in string.punctuation:
        sentence = sentence.replace(punct, '')
    for word in sentence.split():
        if word not in stop_words:
            cleaned.append(word)
    return ' '.join(cleaned).lower()

In [19]:
data['cleaned_facts'] = data.apply(remove_stopwords, axis = 1)

In [20]:
data[['facts', 'cleaned_facts']]

,facts,cleaned_facts
1,<p>Joan Stanley had three children with Peter ...,joan stanley three children peter stanley the ...
2,<p>John Giglio was convicted of passing forged...,john giglio convicted passing forged money ord...
3,"<p>The Idaho Probate Code specified that ""male...",the idaho probate code specified males must pr...
4,"<p>Miller, after conducting a mass mailing cam...",miller conducting mass mailing campaign advert...
5,<p>Ernest E. Mandel was a Belgian professional...,ernest e mandel belgian professional journalis...
...,...,...
3297,<p>For over a century after the Alaska Purchas...,for century alaska purchase 1867 federal gover...
3298,"<p>Refugio Palomar-Santiago, a Mexican nationa...",refugio palomarsantiago mexican national grant...
3299,<p>Tarahrick Terry pleaded guilty to one count...,tarahrick terry pleaded guilty one count posse...
3300,<p>Joshua James Cooley was parked in his picku...,joshua james cooley parked pickup truck side r...


In [21]:
print(data['facts'].iloc[0])
print('----')
print(data['cleaned_facts'].iloc[0])

<p>Joan Stanley had three children with Peter Stanley.  The Stanleys never married, but lived together off and on for 18 years.  When Joan died, the State of Illinois took the children.  Under Illinois law, unwed fathers were presumed unfit parents regardless of their actual fitness and their children became wards of the state.  Peter appealed the decision, arguing that the Illinois law violated the Equal Protection Clause of the Fourteenth Amendment because unwed mothers were not deprived of their children without a showing that they were actually unfit parents.  The Illinois Supreme Court rejected Stanley’s Equal Protection claim, holding that his actual fitness as a parent was irrelevant because he and the children’s mother were unmarried.</p>

----
joan stanley three children peter stanley the stanleys never married lived together 18 years when joan died state illinois took children under illinois law unwed fathers presumed unfit parents regardless actual fitness children became wa

In [67]:
data.iloc[329]

facts                 <p>The MITE Corp, organized under Delaware law...
issue_area                                            Economic Activity
first_party_winner                                                False
cleaned_facts         the mite corp organized delaware laws principa...
Name: 336, dtype: object

In [23]:
final_df = data.drop(['facts'], axis = 1)

In [24]:
lr = LogisticRegression()
dt = DecisionTreeClassifier()
gb = GradientBoostingClassifier()
rf = RandomForestClassifier()
xgb = xgboost.XGBClassifier()
cbt = CatBoostClassifier()
svc = SVC()
models = [lr, dt, gb, rf, xgb, cbt, svc]
names = ['Logistic Regression','DecisionTree','GradientBoosting','RandomForest','XGB','CATBOOST', 'support vector machine']

def evaluate_models(xTrain, yTrain, xVal, yVal, feature = ""):
    xs = [xTrain, yTrain, xVal, yVal]
    random_seed= 0
    scores = []
    for i in range(len(models)):
        model = models[i] 
        model.fit(xTrain, yTrain)
        y_predicted = model.predict(xVal)
        accuracy = accuracy_score(yVal, y_predicted)
        f1score = f1_score(yVal, y_predicted)
        precision = precision_score(yVal, y_predicted)
        recall = recall_score(yVal, y_predicted)
        scores.append({
            'model':names[i] + ' with ' + feature ,
            'model_score':model.score(xTrain, yTrain),
            'accuracy' : accuracy,
            'f1_score' : f1score,
            'precision': precision,
            'recall' : recall
        })
    return pd.DataFrame(scores, columns=['model','model_score','accuracy','f1_score', 'precision', 'recall'])

In [25]:
def get_cv(X, y):
    scores = []
    for i in range(len(models)):
        model = models[i]
        cv = np.mean(np.absolute(cross_val_score(model, X, y, cv = 5)))
        scores.append({'model' : names[i],
                      'CV' : cv})
    return pd.DataFrame(scores, columns = ['model', 'CV'])

In [26]:
X = final_df.reset_index().drop(['index','first_party_winner'], axis = 1)
y = final_df.reset_index()['first_party_winner'].map({True:1, False:0})

In [27]:
for col in X.columns:
    if col != 'cleaned_facts':
        uniq_val = {v : i for i,v in enumerate(np.unique(X[col]))}
        print(uniq_val)
        X[col] = X[col].map(uniq_val)

{'Attorneys': 0, 'Civil Rights': 1, 'Criminal Procedure': 2, 'Due Process': 3, 'Economic Activity': 4, 'Federal Taxation': 5, 'Federalism': 6, 'First Amendment': 7, 'Interstate Relations': 8, 'Judicial Power': 9, 'Miscellaneous': 10, 'Privacy': 11, 'Private Action': 12, 'Unions': 13}


In [28]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.3)

In [73]:
Xtest.iloc[0]

issue_area                                                       3
cleaned_facts    in 2015 markkaya jean gullett montana resident...
Name: 3110, dtype: object

# Using Bag of words approach

In [30]:
cv = CountVectorizer()

In [31]:
bowX = cv.fit_transform(X['cleaned_facts'])

In [32]:
final_df.shape

(3159, 3)

In [33]:
bowX.toarray().shape

(3159, 23984)

In [34]:
bowX = pd.DataFrame(bowX.toarray())

In [35]:
bowX_f = pd.concat([bowX, X.drop(['cleaned_facts'], axis = 1)], axis = 1)

In [36]:
bowX_f

,0,1,2,3,4,5,6,7,8,9,...,23975,23976,23977,23978,23979,23980,23981,23982,23983,issue_area
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [37]:
get_cv(bowX_f, y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarn

Streaming output truncated to the last 5000 lines.
4:	learn: 0.6856298	total: 462ms	remaining: 1m 31s
5:	learn: 0.6841659	total: 541ms	remaining: 1m 29s
6:	learn: 0.6828456	total: 626ms	remaining: 1m 28s
7:	learn: 0.6815424	total: 704ms	remaining: 1m 27s
8:	learn: 0.6802742	total: 779ms	remaining: 1m 25s
9:	learn: 0.6790708	total: 861ms	remaining: 1m 25s
10:	learn: 0.6778863	total: 944ms	remaining: 1m 24s
11:	learn: 0.6767465	total: 1.02s	remaining: 1m 24s
12:	learn: 0.6756713	total: 1.11s	remaining: 1m 24s
13:	learn: 0.6746153	total: 1.18s	remaining: 1m 23s
14:	learn: 0.6736171	total: 1.26s	remaining: 1m 22s
15:	learn: 0.6726372	total: 1.35s	remaining: 1m 22s
16:	learn: 0.6716624	total: 1.43s	remaining: 1m 22s
17:	learn: 0.6706844	total: 1.51s	remaining: 1m 22s
18:	learn: 0.6697556	total: 1.59s	remaining: 1m 22s
19:	learn: 0.6688750	total: 1.67s	remaining: 1m 21s
20:	learn: 0.6680261	total: 1.75s	remaining: 1m 21s
21:	learn: 0.6671533	total: 1.83s	remaining: 1m 21s
22:	learn: 0.666310

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklea

,model,CV
0,Logistic Regression,0.579611
1,DecisionTree,0.563781
2,GradientBoosting,0.627410
3,RandomForest,0.651787
4,XGB,0.639125
5,CATBOOST,0.634062
6,support vector machine,0.656537


In [38]:
bow_train = pd.DataFrame(cv.fit_transform(Xtrain['cleaned_facts']).toarray())
bow_train = pd.concat([bow_train, Xtrain.reset_index().drop(['index','cleaned_facts'], axis = 1)], axis = 1)

bow_test = pd.DataFrame(cv.transform(Xtest['cleaned_facts']).toarray())
bow_test = pd.concat([bow_test, Xtest.reset_index().drop(['index','cleaned_facts'], axis = 1)], axis = 1)

In [39]:
bow_train

,0,1,2,3,4,5,6,7,8,9,...,19804,19805,19806,19807,19808,19809,19810,19811,19812,issue_area
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2206,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2207,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2208,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2209,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7


In [40]:
bow_test

,0,1,2,3,4,5,6,7,8,9,...,19804,19805,19806,19807,19808,19809,19810,19811,19812,issue_area
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
944,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
945,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
946,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [41]:
evaluate_models(bow_train, ytrain, bow_test, ytest, feature = 'Bag of words')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarn

Learning rate set to 0.014457
0:	learn: 0.6914568	total: 88.3ms	remaining: 1m 28s
1:	learn: 0.6897705	total: 162ms	remaining: 1m 20s
2:	learn: 0.6881758	total: 253ms	remaining: 1m 23s
3:	learn: 0.6865972	total: 340ms	remaining: 1m 24s
4:	learn: 0.6850553	total: 420ms	remaining: 1m 23s
5:	learn: 0.6835527	total: 502ms	remaining: 1m 23s
6:	learn: 0.6821030	total: 584ms	remaining: 1m 22s
7:	learn: 0.6807080	total: 664ms	remaining: 1m 22s
8:	learn: 0.6793667	total: 746ms	remaining: 1m 22s
9:	learn: 0.6780647	total: 831ms	remaining: 1m 22s
10:	learn: 0.6767950	total: 910ms	remaining: 1m 21s
11:	learn: 0.6755773	total: 989ms	remaining: 1m 21s
12:	learn: 0.6743436	total: 1.07s	remaining: 1m 21s
13:	learn: 0.6731461	total: 1.16s	remaining: 1m 21s
14:	learn: 0.6720361	total: 1.25s	remaining: 1m 22s
15:	learn: 0.6709409	total: 1.34s	remaining: 1m 22s
16:	learn: 0.6698808	total: 1.42s	remaining: 1m 21s
17:	learn: 0.6688340	total: 1.5s	remaining: 1m 21s
18:	learn: 0.6678420	total: 1.59s	remaining:

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


,model,model_score,accuracy,f1_score,precision,recall
0,Logistic Regression with Bag of words,1.000000,0.579114,0.690937,0.646377,0.742097
1,DecisionTree with Bag of words,1.000000,0.589662,0.689049,0.663077,0.717138
2,GradientBoosting with Bag of words,0.739484,0.633966,0.770351,0.639560,0.968386
3,RandomForest with Bag of words,1.000000,0.632911,0.772846,0.635875,0.985025
4,XGB with Bag of words,0.720036,0.644515,0.780170,0.641631,0.995008
5,CATBOOST with Bag of words,0.776572,0.641350,0.778068,0.640172,0.991681
6,support vector machine with Bag of words,0.825418,0.635021,0.776486,0.634636,1.000000


In [45]:
import pickle as pk

In [47]:
#with open('bow_cv.pk', 'wb') as f:
  #pk.dump(cv, f)

models[4].save_model('model.bin')